In [ ]:
# Import packages required for the project
import geopandas as gpd
import pandas as pd
import requests
import warnings
import folium
import matplotlib
import mapclassify
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# Fetch general information about the Ukraine Refugee Situation
# Source: the United Nations High Commissioner for Refugees (UNHCR).
situation_description = requests.get("https://data2.unhcr.org/population/?widget_id=294522&sv_id=54&population_group=5460")


# Create a Geopandas dataframe containing the UNHCR data.
sd_gpd = gpd.GeoDataFrame(situation_description.json())


# Get the Refugee situation official name and description
situation_name = sd_gpd['situation_view_name'][0]
situation_description = sd_gpd['situation_view_description'][0]


# Print name and description
print("{}:\n\n{}".format(situation_name, situation_description))

In [ ]:
# Fetch the latest refugee counts from the United Nations High Commissioner for Refugees (UNHCR) feed
data = requests.get("https://data2.unhcr.org/population/get/sublocation?widget_id=285457&sv_id=54&population_group=5461&forcesublocation=0&fromDate=1900-01-01")


# Create a Geopandas dataframe containing the UNHCR data
gdf = gpd.GeoDataFrame(data.json())


# Create a pandas dataframe containing information about countries received refugees from Ukraine 
list = []

for i in range(0, len(gdf['data'])):
    list += [[gdf['data'][i]['geomaster_name'], gdf['data'][i]['individuals']]]

df = pd.DataFrame()

for country in list:
    temporary_df = pd.DataFrame([country], columns=['name', 'refugees'])
    df = df.append(temporary_df, ignore_index=True)


# Change data type for the refugees counts from String to Int
df['refugees'] = df['refugees'].astype('int')


# Update the 'Republic of Molvoda' and 'Russian Federation' names to 'Moldova' and 'Russia'
# to make them consistent with the naming convention in the JSON file
# that includes geometries for countries boundaries. 
russia = df['name'].str.contains('Russian Federation')
df.loc[russia, "name"] = 'Russia'
moldova = df['name'].str.contains('Republic of Moldova')
df.loc[moldova, "name"] = 'Moldova'

## Choropleth Map

In [ ]:
# Load a shapefile with national boundaries in to a Geopandas dataframe
countries = gpd.read_file('https://kostya.io/static/data/ukraine_refugee_situation/IPUMSI_world_release2020.zip')


# Change the 'CNTRY_NAME' to just 'name'
countries.rename(columns={'CNTRY_NAME':'name'}, inplace=True)


# Merge datasets containing refugees numbers by country and the countries' geographic boundaries
merged_data = countries.merge(df, on='name')


# Create a Choropleth map using Folium to highlight countries
# recieved Ukranian refugees since the beginning of the conflict on February 24, 2022  

m = folium.Map(location=[48.3794, 31.1656], zoom_start=5)

cpleth = folium.Choropleth(merged_data, data=merged_data,
                  key_on='feature.properties.name',
                           columns=['name', 'refugees'],
                           fill_color="YlOrRd",
                           fill_opacity=0.5,
                           legend_name="Total Refugee influx from Ukraine in neighboring countries", 
                           name="Ukraine Refugee Situation")

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}

info = folium.features.GeoJson(
    merged_data,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['name','refugees'],
        aliases=['Country: ','Refugees: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
m.add_child(info)
m.keep_in_front(info)
cpleth.add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
m

## Refugee Influx by Country Horizontal Bar Chart

In [ ]:
# Get detailed information about the Ukraine Refugee Situation
refugees_counts = requests.get('https://data2.unhcr.org/population/get/sublocation?widget_id=294516&sv_id=54&population_group=5461&forcesublocation=0&fromDate=1900-01-01')


# Create a Geopandas dataframe containing the UNHCR data
rc_gpd = gpd.GeoDataFrame(refugees_counts.json())

# Create a pandas dataframe to store refugees information
rc_df = pd.DataFrame(columns=["Country", "Source", "Last Updated", "Refugees"])


for i in range(0, len(rc_gpd['data'])):
    rc_df = rc_df.append({'Country': rc_gpd['data'][i]['geomaster_name'],
                          'Source': rc_gpd['data'][i]['source'],
                          'Last Updated': rc_gpd['data'][i]['date'],
                          'Refugees': rc_gpd['data'][i]['individuals']},
                         ignore_index=True)

    
# Change data type for the refugees counts from String to Int
rc_df['Refugees'] = rc_df['Refugees'].astype('int')

# plotting Horizontal Bar Chart
rc_df.plot.barh(x='Country', y='Refugees', title='Total Refugee influx from Ukraine in neighboring countries', figsize=(10, 10), color='#de605b', edgecolor="white", linewidth="2")

plt.grid(which='major', axis='x', zorder=-1.0)
plt.rcParams['axes.facecolor']='#222222'


# set the label
plt.xlabel("Refugees (in millions)")

# display the plotted Horizontal Bar Chart
plt.show()

## Total Refugee Infux by Day

In [ ]:
# Get detailed information about the Ukraine Refugee Situation
refugees_by_day = requests.get("https://data2.unhcr.org/population/get/timeseries?widget_id=294518&sv_id=54&population_group=5460&frequency=day&fromDate=1900-01-01")

# Create a Geopandas dataframe containing the UNHCR data
rbd_gpd = gpd.GeoDataFrame(refugees_by_day.json())

# Create a pandas dataframe to store refugees information
rbd_df = pd.DataFrame(columns=["Date", "Unix Timestamp", "Individuals"])
                
                               
for i in range(0, len(rbd_gpd['data'][0])):
    rbd_df = rbd_df.append({'Date': rbd_gpd['data'][0][i]['data_date'],
                            'Unix Timestamp': rbd_gpd['data'][0][i]['unix_timestamp'],
                            'Individuals': rbd_gpd['data'][0][i]['individuals']},
                           ignore_index=True)


# Use an inbuilt style to change the look and feel of the plot
plt.style.use("dark_background")


# setting figure size to 12, 10
plt.figure(figsize=(10, 10))

plt.grid(which='major', axis='y', zorder=-1.0)
plt.rcParams['axes.facecolor']='#222222'

# Label the axes and setting a title
plt.xlabel("Day, #")
plt.ylabel("Refugees")
plt.title(" Refugees from Ukraine in neighboring countries by day")


# plotting the "A" column alone
plt.plot(rbd_df["Individuals"], color='#de605b', linewidth="5")

## Refugee from Ukraine in neighboring countries by month

In [ ]:
# Convert Unix Timestamp to Datetime
rbd_df['DateTime'] = pd.to_datetime(rbd_df['Unix Timestamp'], unit="s")


# Extract name of the month
rbd_df['Month'] = rbd_df['DateTime'].dt.month_name()


# Calculate new refugees counts by day (non-cummulative)
rbd_df['Growth'] = (rbd_df['Individuals'] - rbd_df['Individuals'].shift(1))


# Group data by Month to get aggregate counts for each period
rbd_df_agg = rbd_df.groupby('Month').sum()


# Sort months in chronological order
rbd_df_agg_sorted = rbd_df_agg.sort_index(key=lambda x: pd.to_datetime(x, format="%B"))


# Select 2 columns: Month and Growth (# of refugees)
rbd_df_agg_sorted_select_columns = rbd_df_agg_sorted['Growth']


# Plot a bar chart with a dark backgound
rbd_df_agg_sorted_select_columns.plot(kind="bar", figsize=(10, 10), color="#de605b", edgecolor="white", linewidth="2")

plt.grid(which='major', axis='y', zorder=-1.0)
plt.rcParams['axes.facecolor']='#222222'

# Add a title
plt.title("Refugee from Ukraine in neighboring countries by month")

## Total Refugee influx from Ukraine (pie chart)

In [ ]:
# Assign x and y coordinates
nonnan_countries = rc_df['Country'].unique().tolist()
nonnan_countries_refugees = rc_df['Refugees'].unique().tolist()


# Create the visualization
fig = plt.figure(figsize=(10, 10))
ax = fig.add_axes([1,1,1,1], facecolor="#222222")
colors = ('#de605b', '#de605b', '#de605b', '#de605b', '#de605b', '#de605b', '#de605b')
explode = (0.025, 0.025, 0.025, 0.025, 0.025, 0.025, 0.025)
ax.axis('equal')
ax.pie(nonnan_countries_refugees, labels = nonnan_countries, 
       autopct='%1.2f%%', colors=colors, explode=explode, wedgeprops={"edgecolor":"white",'linewidth': 2, 'antialiased': True})
ax.set_aspect('equal')

# Add a title
plt.title(label="Total Refugee influx from Ukraine in neighboring countries",
          loc="center")

plt.savefig('total_influx_by_country_pie_chart.png')

## Total Refugee influx from Ukraine (donut chart)

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10,10))
ax1.pie(nonnan_countries_refugees, labels = nonnan_countries, 
       autopct='%1.2f%%', colors=colors, explode=explode, wedgeprops={"edgecolor":"white",'linewidth': 2, 'antialiased': True})
#draw circle
centre_circle = plt.Circle((0,0),0.75,fc='#222222', edgecolor="white", linewidth="2")
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  
plt.tight_layout()
# Add a title
plt.title(label="Total Refugee influx from Ukraine in neighboring countries",
          loc="center")
plt.show()